[Completion Prompts Customization](https://docs.llamaindex.ai/en/stable/examples/customization/prompts/completion_prompts/#completion-prompts-customization)

In [ ]:
# MIXTRAL_TEXT_GENERATION_MODEL_REPO = "mistralai/Mixtral-8x7B-Instruct-v0.1"


# import logging
# import sys

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


# import torch

# torch.cuda.is_available()
# print("DEVICE:", "GPU" if torch.cuda.is_available() else "CPU")


# from llama_index.core import Settings
# from llama_index.embeddings.huggingface import (
#     HuggingFaceEmbedding,
# )
# from llama_index.llms.huggingface import HuggingFaceInferenceAPI


# Settings.embed_model = HuggingFaceEmbedding(cache_folder="./tmp/models/")

# Settings.llm = HuggingFaceInferenceAPI(
#     tokenizer_name=MIXTRAL_TEXT_GENERATION_MODEL_REPO,
#     model_name=MIXTRAL_TEXT_GENERATION_MODEL_REPO,
#     device_map="sequential",
# )

import HFH 

In [ ]:
from llama_index.core import PromptTemplate

text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\nUsing"
    " both the context information and also using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you can also"
    " answer the question on your own.\n"
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing both the new"
    " context and your own knowledge, update or repeat the existing answer.\n"
)
refine_template = PromptTemplate(refine_template_str)

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/test/").load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
print(index.as_query_engine().query("what is iris?"))

In [ ]:
print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
    ).query("tell me about Wind flower")
)